# NAVIGATION MODEL USING OPEN ROUTE SERVICE

In [128]:
# The idea behind using the below code is given in the link below
# https://openrouteservice.org/example-avoid-obstacles-while-routing/

In [129]:
#Round 1
# Normal Route

import folium
import pyproj
import requests
from openrouteservice import client
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon


#API KEY
api_key = '5b3ce3597851110001cf62484a0bee2bedfa4902bd8e3af3987cd5df'

ors_client = client.Client(key=api_key)

# profile = ( driving-car, driving-hgv, foot-walking, foot-hiking, cycling-regular,
# cycling-road, cycling-mountain, cycling-electric )

route_normal = ors_client.directions(coordinates=coordinates,
                          profile='driving-car',
                          format='geojson')

def style_function_A(feature):
    return {'opacity': 5, 
            'weight': 3, 
            'color':'RED'} 

Normal_Navigation = folium.Map(location=[41.8781, -87.6798], zoom_start=11)

# add geojson to map
folium.GeoJson(route_normal, name='Normal Route',style_function= style_function_A,overlay=True).add_to(Normal_Navigation)

folium.Marker(list(start_latlng),popup=coordinates[0],icon=folium.Icon(color='orange', icon='home')).add_to(Normal_Navigation)
folium.Marker(list(end_latlng),popup=coordinates[1],icon=folium.Icon(color='black', icon='home')).add_to(Normal_Navigation)



# add layer control to map (allows layer to be turned on or off)
folium.TileLayer('openstreetmap').add_to(Normal_Navigation)
folium.TileLayer('Stamen Terrain').add_to(Normal_Navigation)
folium.TileLayer('Stamen Toner').add_to(Normal_Navigation)
folium.TileLayer('Stamen Water Color').add_to(Normal_Navigation)
folium.TileLayer('cartodbpositron').add_to(Normal_Navigation)
folium.TileLayer('cartodbdark_matter').add_to(Normal_Navigation)
folium.LayerControl().add_to(Normal_Navigation)

# display map
Normal_Navigation

In [151]:
# distance in Miles and duration in Hours

print(route_normal['features'][0]['properties']['segments'][0]['distance']*0.000621371, 'miles')
print(route_normal['features'][0]['properties']['segments'][0]['duration']*0.000277778, 'hours\n')

9.5431400922 miles
0.2785557784 hours



In [152]:
# Distances are in meters
# Timings are in seconds
print('directions')
for index, i in enumerate(route_normal['features'][0]['properties']['segments'][0]['steps']):
    print(index+1, i, '\n')

directions
1 {'distance': 156.7, 'duration': 29.0, 'type': 11, 'instruction': 'Head south on North Halsted Street', 'name': 'North Halsted Street', 'way_points': [0, 10]} 

2 {'distance': 101.5, 'duration': 14.6, 'type': 0, 'instruction': 'Turn left onto West Lake Street', 'name': 'West Lake Street', 'way_points': [10, 14]} 

3 {'distance': 312.9, 'duration': 38.3, 'type': 13, 'instruction': 'Keep right', 'name': '-', 'way_points': [14, 25]} 

4 {'distance': 99.7, 'duration': 8.9, 'type': 6, 'instruction': 'Continue straight onto Kennedy Expressway, I 90, I 94', 'name': 'Kennedy Expressway, I 90, I 94', 'way_points': [25, 27]} 

5 {'distance': 2242.1, 'duration': 161.8, 'type': 6, 'instruction': 'Continue straight onto Kennedy Expressway, I 90, I 94', 'name': 'Kennedy Expressway, I 90, I 94', 'way_points': [27, 50]} 

6 {'distance': 2103.5, 'duration': 75.7, 'type': 6, 'instruction': 'Continue straight onto Dan Ryan Expressway, I 90, I 94', 'name': 'Dan Ryan Expressway, I 90, I 94', 'w

## Navigation to avoid risk areas

In [153]:
#Getting the risky locations in the clusters that are closer to the start and end point

Risk_factor = [[LONGITUDE, LATITUDE] for LONGITUDE, LATITUDE in zip(Risky_Areas['LONGITUDE'],Risky_Areas['LATITUDE'])]

In [154]:
# Creating a polygon Perimeter using the risk areas near the navigation area

import folium
import pyproj
import requests 
from openrouteservice import client
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon


#API KEY
api_key = '5b3ce3597851110001cf62484a0bee2bedfa4902bd8e3af3987cd5df'

ors_client = client.Client(key=api_key)

def risk_buffer_polygon(point_in, resolution=10, radius=5):
    convert = pyproj.Transformer.from_crs("epsg:4326", 'epsg:32632')  # epsg:4326 to epsg:32632
    convert_back = pyproj.Transformer.from_crs('epsg:32632', "epsg:4326")  #  epsg:32632 to epsg:4326
    point_in_proj = convert.transform(*point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)  # 10 m buffer

    # Iterate over all points in buffer and build polygon
    RISK_POLYGON = []
    for point in point_buffer_proj.exterior.coords:
        RISK_POLYGON.append(convert_back.transform(*point)) # Transform back to epsg:4326
    
    return RISK_POLYGON

In [170]:
#Folium (lat,long)

#open_street(long,lat)

map_parameters = {'tiles': 'openstreetmap',
              'location': ([41.881832, -87.623177]),
              'zoom_start': 11}
risk_area_location = folium.Map(**map_parameters)

# Populate a construction site buffer polygon list
RISK_POLYGON = []
for risk in Risk_factor:
    site_coords = risk
    folium.features.Marker(list(reversed(site_coords)),
                           popup='Risk_Area<br>{0}'.format(site_coords)).add_to(risk_area_location)

    # Create buffer polygons around construction sites with 10 m radius and low resolution
    risk_poly_cords = risk_buffer_polygon(site_coords,
                                             resolution=1,  # low resolution to keep polygons lean
                                             radius=5)
    RISK_POLYGON.append(risk_poly_cords)

    risk_poly_cords = [(y, x) for x, y in risk_poly_cords]  # Reverse coords for folium/Leaflet
    folium.vector_layers.Polygon(locations=risk_poly_cords,
                                 color='#ffd699',
                                 fill_color='#ffd699',
                                 fill_opacity=0.2,
                                 weight=3).add_to(risk_area_location)
folium.TileLayer('openstreetmap').add_to(risk_area_location)
folium.TileLayer('Stamen Terrain').add_to(risk_area_location)
folium.TileLayer('Stamen Toner').add_to(risk_area_location)
folium.TileLayer('Stamen Water Color').add_to(risk_area_location)
folium.TileLayer('cartodbpositron').add_to(risk_area_location)
folium.TileLayer('cartodbdark_matter').add_to(risk_area_location)
folium.LayerControl().add_to(risk_area_location)
    
risk_area_location

In [174]:
import folium
import pyproj
import requests 
from openrouteservice import client
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon


def style_function(color):
    return lambda feature: dict(color=color,
                                weight=3,
                                opacity=0.5)



map_parameters.update({'location': ([41.881832, -87.623177]),
                   'zoom_start': 11})
Desired_Nagiation_System = folium.Map(**map_parameters)


# Using the Normal Route Map which was done before and comparing with the risk free route

def style_function_B(feature):
    return {'opacity': 5, 
            'weight': 3, 
            'color':'RED'}

folium.features.GeoJson(data=route_normal,
                        name='Normal Route',
                        style_function=style_function_B,
                        overlay=True).add_to(Desired_Nagiation_System)


# Eliminating Risky areas which are not in the immediate surrounding of the route of interest
# With Buffer value of 0.009 degrees covers area aroud


def style_function_D(feature):
    return {'opacity': 5, 
            'weight': 3, 
            'color':'GREEN'}


route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.009)
folium.features.GeoJson(data=geometry.mapping(route_buffer),
                        name='Route Buffer',
                        style_function=style_function_D,
                        overlay=True).add_to(Desired_Nagiation_System)



# Plot which construction sites fall into the buffer Polygon
sites_buffer_poly = []
for RISK_POLYGON in RISK_POLYGON:
    poly = Polygon(RISK_POLYGON)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(Desired_Nagiation_System)
        sites_buffer_poly.append(poly)



# Route which avoids accident prone areas

# Request normal route between appropriate locations without construction sites
request_params = {'coordinates': list(coordinates),
                  'format_out': 'geojson',
                  'profile': 'driving-car',
                  'preference': 'shortest',
                  'instructions': 'true',}

request_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))}
route_detour = ors_client.directions(**request_params)

def style_function_C(feature):
    return {'opacity': 5, 
            'weight': 3, 
            'color':'BLUE'}


route_detour = ors_client.directions(**request_params)
folium.features.GeoJson(data=route_detour,
                        name='Risk Free Route',
                        style_function=style_function_C,
                        overlay=True).add_to(Desired_Nagiation_System)


folium.Marker(list(start_latlng),popup=coordinates[0],icon=folium.Icon(color='orange', icon='home'),color="blue").add_to(Desired_Nagiation_System)
folium.Marker(list(end_latlng),popup=coordinates[1],icon=folium.Icon(color='black', icon='home'),color="red").add_to(Desired_Nagiation_System)
    

        
folium.TileLayer('openstreetmap').add_to(Desired_Nagiation_System)
folium.TileLayer('Stamen Terrain').add_to(Desired_Nagiation_System)
folium.TileLayer('Stamen Toner').add_to(Desired_Nagiation_System)
folium.TileLayer('Stamen Water Color').add_to(Desired_Nagiation_System)
folium.TileLayer('cartodbpositron').add_to(Desired_Nagiation_System)
folium.TileLayer('cartodbdark_matter').add_to(Desired_Nagiation_System)
folium.LayerControl().add_to(Desired_Nagiation_System)


Desired_Nagiation_System

## Normal Route vs Detour Route

In [157]:

# Normal Route


# distance in Miles and duration in Hours

print("The total distance travelled in the normal route is",route_normal['features'][0]['properties']['segments'][0]['distance']*0.000621371, 'miles')
print("The total Time Taken to travel in the normal route is",route_normal['features'][0]['properties']['segments'][0]['duration']*0.000277778*60, 'Minutes\n')



# Distances are in meters
# Timings are in seconds
print('DIRECTIONS: \n')
for index, i in enumerate(route_normal['features'][0]['properties']['segments'][0]['steps']):
    print(index+1, i, '\n')

The total distance travelled in the normal route is 9.5431400922 miles
The total Time Taken to travel in the normal route is 16.713346704 Minutes

DIRECTIONS: 

1 {'distance': 156.7, 'duration': 29.0, 'type': 11, 'instruction': 'Head south on North Halsted Street', 'name': 'North Halsted Street', 'way_points': [0, 10]} 

2 {'distance': 101.5, 'duration': 14.6, 'type': 0, 'instruction': 'Turn left onto West Lake Street', 'name': 'West Lake Street', 'way_points': [10, 14]} 

3 {'distance': 312.9, 'duration': 38.3, 'type': 13, 'instruction': 'Keep right', 'name': '-', 'way_points': [14, 25]} 

4 {'distance': 99.7, 'duration': 8.9, 'type': 6, 'instruction': 'Continue straight onto Kennedy Expressway, I 90, I 94', 'name': 'Kennedy Expressway, I 90, I 94', 'way_points': [25, 27]} 

5 {'distance': 2242.1, 'duration': 161.8, 'type': 6, 'instruction': 'Continue straight onto Kennedy Expressway, I 90, I 94', 'name': 'Kennedy Expressway, I 90, I 94', 'way_points': [27, 50]} 

6 {'distance': 2103.

In [158]:

# Alternative Risk Free Route


# distance in Miles and duration in Hours

print("The total distance travelled in the detour route is",route_detour['features'][0]['properties']['segments'][0]['distance']*0.000621371, 'miles')
print("The total Time Taken to travel in the detour route is",route_detour['features'][0]['properties']['segments'][0]['duration']*0.000277778*60, 'Minutes\n')



# Distances are in meters
# Timings are in seconds
print('DIRECTIONS: \n')
for index, i in enumerate(route_detour['features'][0]['properties']['segments'][0]['steps']):
    print(index+1, i, '\n')

The total distance travelled in the detour route is 8.2608788966 miles
The total Time Taken to travel in the detour route is 31.920025536000004 Minutes

DIRECTIONS: 

1 {'distance': 4435.5, 'duration': 597.6, 'type': 11, 'instruction': 'Head south on North Halsted Street', 'name': 'North Halsted Street', 'way_points': [0, 189]} 

2 {'distance': 107.4, 'duration': 16.6, 'type': 1, 'instruction': 'Turn right onto South Archer Avenue', 'name': 'South Archer Avenue', 'way_points': [189, 192]} 

3 {'distance': 650.8, 'duration': 70.9, 'type': 5, 'instruction': 'Turn slight right onto South Archer Avenue', 'name': 'South Archer Avenue', 'way_points': [192, 208]} 

4 {'distance': 45.7, 'duration': 11.0, 'type': 0, 'instruction': 'Turn left onto South Throop Street', 'name': 'South Throop Street', 'way_points': [208, 211]} 

5 {'distance': 256.1, 'duration': 46.1, 'type': 1, 'instruction': 'Turn right', 'name': '-', 'way_points': [211, 229]} 

6 {'distance': 181.6, 'duration': 32.7, 'type': 0,